### Imports

Using the NPL 2024a environment

In [1]:
import dask
import geocat.comp as gc
import matplotlib.pyplot as plt
import metpy
import numpy as np
import xarray as xr
import xesmf as xe
from functools import partial

In [2]:
from dask.distributed import performance_report

### Spin up a dask cluster

"The cores and memory parameters provide information to Dask, while the resource_spec provides information to PBS."

In [3]:
from dask.distributed import Client
from dask_jobqueue import PBSCluster

# Cluster specification
cluster = PBSCluster(
    # Basic job directives
    job_name = "test_dask_interp",
    queue = "casper",
    walltime = "2:00:00",
    account = "NVST0001",
    log_directory = "dask-logs",
    # These settings impact the resources assigned to the job
    cores = 1,
    memory = "4GiB",
    resource_spec = "select=1:ncpus=1:mem=4GB",
    # These settings define the resources assigned to a worker
    processes = 1,
    # This controls where Dask will write data to disk if memory is exhausted
    local_directory = "$TMPDIR",
    # This specifies which network interface the cluster will use
    interface = "ext"
)

# Scale up
cluster.scale(4)

# Setup your client
client = Client(cluster)

/glade/u/apps/opt/conda/envs/npl-2024a/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36777 instead
  warnings.warn(


In [4]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/anissaz/proxy/36777/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/anissaz/proxy/36777/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.98:40417,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/anissaz/proxy/36777/status,Total threads: 0
Started: Just now,Total memory: 0 B


### Paths

In [5]:
input_path = "/glade/campaign/collections/rda/data/ds633.6/e5.oper.an.ml/"
output_path = "/glade/derecho/scratch/anissaz/era5_processed/"

### Calculate the zonal mean fluxes

In [6]:
var = ("u", "v", "t", "w", "sp")
varinfile = ("U", "V", "T", "W", "SP")
phisera5 = xr.open_dataset("/glade/u/home/islas/python/ERA5interp/phis/ERA5_phis.nc")
phisera5 = phisera5.rename({"g4_lon_3": "longitude", "g4_lat_2": "latitude"})

In [7]:
# create a regridder object from ERA5 to CAM6 it looks like
# warning is because of the use of conservative regridding (still need to look into details)

wgtfile = output_path + "wgtfile.nc"
phis_cam6 = xr.open_dataset("/glade/u/home/islas/python/ERA5interp/phis/CAM6_phis.nc").isel(time=0)
grid_out = xr.Dataset({"lat": (["lat"], phis_cam6.lat.values)}, {"lon": (["lon"], phis_cam6.lon.values)})
regridder = xe.Regridder(
    phisera5,
    grid_out,
    "conservative",
    reuse_weights=False,
    filename=wgtfile,
)

phisera5_rg = regridder(phisera5)

/glade/u/apps/opt/conda/envs/npl-2024a/lib/python3.11/site-packages/xesmf/backend.py:56: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')


In [8]:
phisera5_rg = regridder(phisera5).compute()

In [9]:
def preprocessor(ds):
    ds = ds.loc[{"time": ds.time.dt.hour.isin([0, 6, 12, 18])}]
    return ds

In [10]:
# open some ERA5 data from campaign storage
dat = xr.open_mfdataset(
    input_path + "/202101/e5.oper.an.ml." + "*_[u,v,t,w,s]*." + "*20210101*.nc",
    coords="minimal",
    join="override",
    decode_times=True,
    use_cftime=True,
    parallel=True,
    data_vars="minimal",
    compat="override",
    preprocess=partial(preprocessor),
)

dat = dat.rename(level="plev")

# convert vertical velocity from Pa/s to m/s
g = 9.81
rgas = 287.058
pfull = dat.a_model + dat.b_model * dat.SP
dat["W"] = -1.0 * dat.W / (g * (pfull / (rgas * dat.T)))

# convert t to theta
dat["T"] = dat.T * ((pfull / 100.0) / 1000.0) ** (-2.0 / 7.0)


dat_rg = regridder(dat)

pinterp = dat.a_model + 1e5 * dat.b_model

tbot = dat_rg.T.isel(plev=dat_rg.plev.size - 1)

In [11]:
dat_rg

<xarray.Dataset>
Dimensions:     (time: 4, lat: 192, lon: 288, plev: 137, half_level: 138)
Coordinates:
  * time        (time) object 2021-01-01 00:00:00 ... 2021-01-01 18:00:00
  * half_level  (half_level) int32 1 2 3 4 5 6 7 ... 132 133 134 135 136 137 138
  * plev        (plev) int32 1 2 3 4 5 6 7 8 ... 130 131 132 133 134 135 136 137
  * lat         (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon         (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Data variables:
    SP          (time, lat, lon) float32 dask.array<chunksize=(1, 192, 288), meta=np.ndarray>
    T           (time, plev, lat, lon) float64 dask.array<chunksize=(1, 137, 192, 288), meta=np.ndarray>
    U           (time, plev, lat, lon) float32 dask.array<chunksize=(1, 137, 192, 288), meta=np.ndarray>
    V           (time, plev, lat, lon) float32 dask.array<chunksize=(1, 137, 192, 288), meta=np.ndarray>
    W           (time, plev, lat, lon) float64 dask.array<chunksize=(1, 137, 192, 288), meta=np.ndarray>
Attributes:
    regrid_method:  conservative

In [12]:
dat

<xarray.Dataset>
Dimensions:     (time: 4, latitude: 640, longitude: 1280, plev: 137,
                 half_level: 138)
Coordinates:
  * latitude    (latitude) float64 89.78 89.51 89.23 ... -89.23 -89.51 -89.78
  * longitude   (longitude) float64 0.0 0.2812 0.5625 ... 359.2 359.4 359.7
  * time        (time) object 2021-01-01 00:00:00 ... 2021-01-01 18:00:00
  * half_level  (half_level) int32 1 2 3 4 5 6 7 ... 132 133 134 135 136 137 138
  * plev        (plev) int32 1 2 3 4 5 6 7 8 ... 130 131 132 133 134 135 136 137
Data variables:
    SP          (time, latitude, longitude) float32 dask.array<chunksize=(1, 435, 870), meta=np.ndarray>
    utc_date    (time) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    weight      (latitude) float64 dask.array<chunksize=(640,), meta=np.ndarray>
    zero        (latitude) float64 dask.array<chunksize=(640,), meta=np.ndarray>
    T           (time, plev, latitude, longitude) float64 dask.array<chunksize=(1, 137, 435, 870), meta=np.ndarray>
    a_half      (half_level) float64 dask.array<chunksize=(138,), meta=np.ndarray>
    a_model     (plev) float64 dask.array<chunksize=(137,), meta=np.ndarray>
    b_half      (half_level) float64 dask.array<chunksize=(138,), meta=np.ndarray>
    b_model     (plev) float64 dask.array<chunksize=(137,), meta=np.ndarray>
    U           (time, plev, latitude, longitude) float32 dask.array<chunksize=(1, 137, 640, 1280), meta=np.ndarray>
    V           (time, plev, latitude, longitude) float32 dask.array<chunksize=(1, 137, 640, 1280), meta=np.ndarray>
    W           (time, plev, latitude, longitude) float64 dask.array<chunksize=(1, 137, 435, 870), meta=np.ndarray>
Attributes: (12/18)
    RDA_DATASET:                            ds633.6
    RDA_DATASET_URL:                        https:/rda.ucar.edu/datasets/ds63...
    RDA_DATASET_DOI:                        DOI: 10.5065/XV5R-5344
    RDA_DATASET_GROUP:                      ERA5 atmospheric model level anal...
    DATA_SOURCE:                            Downloaded by NCAR/CISL/DECS from...
    DOWNLOAD_SPECIALIST:                    Chi-Fan Shih, NCAR/CISL/DECS, chi...
    ...                                     ...
    NETCDF_COMPRESSION:                     Precision-preserving compression ...
    CONVERSION_AND_COMPRESSION_PLATFORM:    cheyenne node employed: Linux r13...
    CONVERSION_AND_COMPRESSION_DATE:        Sat Jan 14 13:30:40 MST 2023
    CONVERSION_AND_COMPRESSION_SPECIALIST:  Dave Stepaniak, NCAR/CISL/DECS, d...
    history:                                Sat Jan 14 13:30:42 2023: ncks -4...
    NCO:                                    netCDF Operators version 5.0.3 (H...

In [13]:
remote_dat_rg = {
    "U": client.scatter(dat_rg.U),
    "SP": client.scatter(dat_rg.SP),
}

remote_dat = {
    "a_model": client.scatter(dat.a_model),
    "b_model": client.scatter(dat.b_model),
}

In [14]:
remote_dat["a_model"]

<Future: finished, type: xarray.core.dataarray.DataArray, key: DataArray-97c65921d88d8a84b5babaf92be7eaf9>

In [17]:
%%prun

#with performance_report(filename="dask-report-persisted.html"):
U_plev = client.submit( interp_hybrid_to_pressure,
    remote_dat_rg["U"],
    remote_dat_rg["SP"],
    remote_dat["a_model"],
    remote_dat["b_model"],
    p0=1,
    new_levels=pinterp,
    method="log",
    lev_dim="plev",
    extrapolate=True,
    variable="other",
    t_bot=tbot,
    phi_sfc=phisera5_rg.Z_GDS4_SFC,
)


#U_plev.compute()

/glade/u/apps/opt/conda/envs/npl-2024a/lib/python3.11/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 30.40 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


         84054 function calls (82955 primitive calls) in 0.080 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.031    0.031    0.031    0.031 {method 'send' of '_socket.socket' objects}
       14    0.012    0.001    0.019    0.001 cloudpickle.py:329(_find_imported_submodules)
    47293    0.006    0.000    0.006    0.000 {method 'startswith' of 'str' objects}
      784    0.002    0.000    0.007    0.000 dis.py:423(_get_instructions_bytes)
  118/116    0.002    0.000    0.008    0.000 {built-in method builtins.repr}
        3    0.002    0.001    0.002    0.001 {built-in method _pickle.dumps}
        2    0.001    0.001    0.030    0.015 {function Pickler.dump at 0x14ff64d5e700}
     1568    0.001    0.000    0.001    0.000 dis.py:594(_unpack_opargs)
   589/10    0.001    0.000    0.003    0.000 arrayprint.py:789(recurser)
      792    0.001    0.000    0.001    0.000 {built-in method numpy.core._multiarray_

In [18]:
%%prun
U_plev.result()

         74 function calls in 0.479 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        4    0.479    0.120    0.479    0.120 {method 'acquire' of '_thread.lock' objects}
        1    0.000    0.000    0.479    0.479 {built-in method builtins.exec}
        1    0.000    0.000    0.479    0.479 utils.py:346(sync)
        1    0.000    0.000    0.479    0.479 utils.py:375(sync)
        1    0.000    0.000    0.479    0.479 client.py:300(result)
        1    0.000    0.000    0.000    0.000 {method 'send' of '_socket.socket' objects}
        1    0.000    0.000    0.479    0.479 threading.py:295(wait)
        1    0.000    0.000    0.000    0.000 threading.py:243(__init__)
        1    0.000    0.000    0.479    0.479 threading.py:611(wait)
        1    0.000    0.000    0.000    0.000 gen.py:177(coroutine)
        1    0.000    0.000    0.000    0.000 functools.py:35(update_wrapper)
        1    0.000    0.000    0.000   

In [ ]:
# U_plev.result().data.dask.visualize("no_distribute.svg")

In [19]:
U_plev.result().data.dask

HighLevelGraph with 63 layers.
<dask.highlevelgraph.HighLevelGraph object at 0x14ff18573810>
 0. original-array-b0dff87929aa97289ca88795dea8bb80
 1. array-b0dff87929aa97289ca88795dea8bb80
 2. original-open_dataset-U-d2c8a9ae4b0b4826f9047e38dff54307
 3. open_dataset-U-d2c8a9ae4b0b4826f9047e38dff54307
 4. getitem-81bef82245b0f54dc82accd3efb73ff1
 5. original-open_dataset-U-d7ebbf21e26d2dad5e8170a5e46aec24
 6. open_dataset-U-d7ebbf21e26d2dad5e8170a5e46aec24
 7. getitem-8f643d62cc47e7dc4b5acfa0c4ad2b51
 8. original-open_dataset-U-3d636f3dc817f066c628c0c7e3a86664
 9. open_dataset-U-3d636f3dc817f066c628c0c7e3a86664
 10. getitem-8b9065b2561b823cfea6c352d0d2c080
 11. original-open_dataset-U-6fe2247f2469bc68edff085044ce0b3c
 12. open_dataset-U-6fe2247f2469bc68edff085044ce0b3c
 13. getitem-8cd87ede6e83a94725fe9633dcac7103
 14. concatenate-fd75c62934ec4f3a8e7db7c1de75f15a
 15. tensordot-28f9a3c9a7c10580f6c8286ce90545e6
 16. sum-118b7efd12260d0c3968c831009f87e2
 17. sum-aggregate-560ef40de03f1f59bf473afbb18fce45
 18. astype-74263322a653a00e82f748470fc4a717
 19. getitem-5220e70dbb54a08a27a4d7f1bacb490d
 20. getitem-c8b895e3b5fe3004e65ae29fda746816
 21. original-array-a1a8169e61de6b8c8151f220e2e64006
 22. array-a1a8169e61de6b8c8151f220e2e64006
 23. original-open_dataset-SP-1369c32108ea1cc3d8b33cab982ce8ec
 24. open_dataset-SP-1369c32108ea1cc3d8b33cab982ce8ec
 25. getitem-b320d9ad91f5d332110137dcd787f024
 26. original-open_dataset-SP-9a2cb3f621ed95568128ed86055484ad
 27. open_dataset-SP-9a2cb3f621ed95568128ed86055484ad
 28. getitem-849f3e2785984efaf53b79d7500c20fb
 29. original-open_dataset-SP-66cf8457ed3a9a6bf30a9e6b2bd2d301
 30. open_dataset-SP-66cf8457ed3a9a6bf30a9e6b2bd2d301
 31. getitem-3256c9e0127d6d42fb931e17e60875c6
 32. original-open_dataset-SP-a74bea1921efe61aefe26d9e45d600e4
 33. open_dataset-SP-a74bea1921efe61aefe26d9e45d600e4
 34. getitem-cc656098fb98bbdfa7b17a7d54c83fbf
 35. concatenate-d9c15e55d2191172d829ff9cddc4b062
 36. tensordot-93dba4bcd40be3687963597e9c93cf15
 37. sum-6abf167f27096a445a340a03798f7dcd
 38. sum-aggregate-bf556c4df51ac66d84f717a921249364
 39. astype-179a8707f44cf377bc234bf22bff74c6
 40. getitem-c32b4fc9b12d059e5781603185aa6502
 41. original-open_dataset-b_model-6252af2c7f292b6521c787b2f2e9f18b
 42. open_dataset-b_model-6252af2c7f292b6521c787b2f2e9f18b
 43. mul-402a2d662e63dcc16078af05a4a82e7b
 44. getitem-ac7e517035085d2cbfa04c2bd97979fc
 45. transpose-6a6e55ff2eb4c62aa1f57c2872cc4ee3
 46. mul-64b6c987e19e0bb1d7a64048892d47be
 47. original-open_dataset-a_model-6252af2c7f292b6521c787b2f2e9f18b
 48. open_dataset-a_model-6252af2c7f292b6521c787b2f2e9f18b
 49. add-45963a6285cae3bad9b2c16a3dadf440
 50. finalize-840b1d2e-61f1-48d2-a888-fbe42ea2274e
 51. mul-be2a9b0ea0d1ec15c043a6aed0c9ff47
 52. getitem-0fd577301670caa5ec2cf0d95d48edbd
 53. transpose-44ded35cc9add9bebdda81bd28aa5e2f
 54. add-aadcdebb9b5d6fca0953834961f62ea7
 55. transpose-9b0e97b1044748b4108f2aa35e54b933
 56. _vertical_remap-1038f1dbbc057155c556c7d8fa1d7eef
 57. getitem-4239c9985169584a1bb96f4369b8e72c
 58. getitem-739b3a9539c0dd5905473a04f525fbde
 59. array-cef3489b00a7c268b055a555984ba710
 60. less_equal-8767baba42c8047415581dd0cb98a9ee
 61. transpose-20665fb1aaa34255f8fb16e07330dc7c
 62. where-7753bc5134c68a49f0580bbe0c97d834

In [20]:
%%prun

U_plev = interp_hybrid_to_pressure(
    dat_rg["U"],
    dat_rg["SP"],
    dat["a_model"],
    dat["b_model"],
    p0=1,
    new_levels=pinterp,
    method="log",
    lev_dim="plev",
    extrapolate=True,
    variable="other",
    t_bot=tbot,
    phi_sfc=phisera5_rg.Z_GDS4_SFC,
)

/glade/derecho/scratch/anissaz/tmp/ipykernel_68015/4223488145.py:467: DeprecationWarning: Supplying chunks as dimension-order tuples is deprecated. It will raise an error in the future. Instead use a dict with dimension names as keys.
  pressure = pressure.chunk(data.chunks)


         127865 function calls (126532 primitive calls) in 0.298 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        9    0.201    0.022    0.201    0.022 {method 'acquire' of '_thread.lock' objects}
      355    0.008    0.000    0.009    0.000 {built-in method io.open}
    39187    0.005    0.000    0.005    0.000 {built-in method builtins.getattr}
      354    0.005    0.000    0.009    0.000 __init__.py:343(<setcomp>)
    77/74    0.002    0.000    0.004    0.000 utils.py:27(meta_from_array)
        7    0.002    0.000    0.002    0.000 {method 'send' of '_socket.socket' objects}
        6    0.002    0.000    0.002    0.000 {built-in method _pickle.dumps}
      354    0.002    0.000    0.002    0.000 __init__.py:340(<setcomp>)
7699/7696    0.001    0.000    0.004    0.000 {built-in method builtins.isinstance}
      354    0.001    0.000    0.016    0.000 __init__.py:844(read_text)
      355    0.001    0.000    0.0

In [21]:
%%prun
U_plev = gc.interp_hybrid_to_pressure(
    dat_rg["U"],
    dat_rg["SP"],
    dat["a_model"],
    dat["b_model"],
    p0=1,
    new_levels=pinterp,
    method="log",
    lev_dim="plev",
    extrapolate=True,
    variable="other",
    t_bot=tbot,
    phi_sfc=phisera5_rg.Z_GDS4_SFC,
)

         208706076 function calls (208621248 primitive calls) in 94.032 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   950232   29.638    0.000   54.351    0.000 highlevelgraph.py:508(__getitem__)
 97912561   18.264    0.000   18.264    0.000 highlevelgraph.py:325(__getitem__)
    81494   12.911    0.000   24.334    0.000 {method 'update' of 'dict' objects}
     1084   11.647    0.011   11.647    0.011 {method 'acquire' of '_thread.lock' objects}
 19623062    6.781    0.000   13.853    0.000 blockwise.py:491(__getitem__)
 19639232    5.243    0.000    7.237    0.000 blockwise.py:453(_dict)
 46234712    3.771    0.000    3.989    0.000 <frozen _collections_abc>:835(__iter__)
19762228/19761531    1.871    0.000    1.906    0.000 {built-in method builtins.hasattr}
      137    0.405    0.003   79.539    0.581 slicing.py:1515(setitem_array)
      828    0.371    0.000    0.371    0.000 {method 'send' of '_socket.socket' obj

### GeoCAT function of interest

In [16]:
import typing

import cf_xarray
import metpy.interpolate
import numpy as np
import xarray as xr

supported_types = typing.Union[xr.DataArray, np.ndarray]

__pres_lev_mandatory__ = np.array([
    1000, 925, 850, 700, 500, 400, 300, 250, 200, 150, 100, 70, 50, 30, 20, 10,
    7, 5, 3, 2, 1
]).astype(np.float32)  # Mandatory pressure levels (mb)
__pres_lev_mandatory__ = __pres_lev_mandatory__ * 100.0  # Convert mb to Pa


def _func_interpolate(method='linear'):
    """Define interpolation function."""

    if method == 'linear':
        func_interpolate = metpy.interpolate.interpolate_1d
    elif method == 'log':
        func_interpolate = metpy.interpolate.log_interpolate_1d
    else:
        raise ValueError(f'Unknown interpolation method: {method}. '
                         f'Supported methods are: "log" and "linear".')

    return func_interpolate


def _pressure_from_hybrid(psfc, hya, hyb, p0=100000.):
    """Calculate pressure at the hybrid levels."""

    # p(k) = hya(k) * p0 + hyb(k) * psfc

    # This will be in Pa
    return hya * p0 + hyb * psfc


def _pre_interp_multidim(
    data_in: xr.DataArray,
    cyclic: bool,
    missing_val,
):
    """Helper Function: Handling missing data functionality and adding cyclic
    point if required.

    Parameters
    ----------
    data_in : :class:`xarray.DataArray`
        The data on which to operate

    cyclic : :class:`bool`
        Determines if cyclic point should be added or not.
        If true then add point, else do nothing.

    missing_val : int, float, optional
        Provides an alternative to NaN

    Returns
    -------
    data_in : :class:`xarray.DataArray`
       The data input with cyclic points added (if cyclic is true)
       and missing_val values replaced with np.nan

    Notes
    -------
    """
    # replace missing_val with np.nan
    if missing_val is not None:
        data_in = xr.DataArray(np.where(data_in.values == missing_val, np.nan,
                                        data_in.values),
                               dims=data_in.dims,
                               coords=data_in.coords)

    # add cyclic points and create new data array
    if cyclic:
        padded_data = np.pad(data_in.values, ((0, 0), (1, 1)), mode='wrap')
        padded_longitudes = np.pad(data_in.coords[data_in.dims[-1]], (1, 1),
                                   mode='wrap')
        padded_longitudes[0] -= 360
        padded_longitudes[-1] += 360

        data_in = xr.DataArray(
            padded_data,
            coords={
                data_in.dims[-2]: data_in.coords[data_in.dims[-2]].values,
                data_in.dims[-1]: padded_longitudes,
            },
            dims=data_in.dims,
        )

    return data_in


def _post_interp_multidim(data_in, missing_val):
    """Helper Function: Handling missing data functionality.

    Parameters
    ----------
    data_in : :class:`xarray.DataArray`
        The data on which to operate

    missing_val : int, float, optional
         Provides an alternative to NaN

    Returns
    -------
    data_in : :class:`xarray.DataArray`
       The data input with np.nan values replaced with missing_val
    """
    if missing_val is not None:
        data_in = xr.DataArray(np.where(np.isnan(data_in.values), missing_val,
                                        data_in.values),
                               dims=data_in.dims,
                               coords=data_in.coords)

    return data_in


def _sigma_from_hybrid(psfc, hya, hyb, p0=100000.):
    """Calculate sigma at the hybrid levels."""

    # sig(k) = hya(k) * p0 / psfc + hyb(k)

    # This will be in Pa
    return hya * p0 / psfc + hyb


def _vertical_remap(func_interpolate, new_levels, xcoords, data, interp_axis=0):
    """Execute the defined interpolation function on data."""

    return func_interpolate(new_levels, xcoords, data, axis=interp_axis)


def _temp_extrapolate(data, lev_dim, lev, p_sfc, ps, phi_sfc):
    r"""This helper function extrapolates temperature below ground using the
    ECMWF formulation described in `Vertical Interpolation and Truncation of
    Model-Coordinate Data <http://dx.doi.org/10.5065/D6HX19NH>`__ by Trenberth,
    Berry, & Buja [NCAR/TN-396, 1993]. Specifically equation 16 is used:

    .. math::
        T = T_* \left( 1 + \alpha ln \frac{p}{p_s} + \frac{1}{2}\left( \alpha ln \frac{p}{p_s} \right)^2 + \frac{1}{6} \left( \alpha ln \frac{p}{p_s} \right)^3 \right)

    Parameters
    ----------
    data: :class:`xarray.DataArray`
        The temperature at the lowest level of the model.

    lev_dim: str
        The name of the vertical dimension.

    lev: int
        The pressure levels of interest. Must be in the same units as ``ps`` and ``p_sfc``

    p_sfc: :class:`xarray.DataArray`
        The pressure at the lowest level of the model. Must be in the same units as ``lev`` and ``ps``

    ps: :class:`xarray.DataArray`
        An array of surface pressures. Must be in the same units as ``lev`` and ``p_sfc``

    phi_sfc: :class:`xarray.DataArray`
        The geopotential at the lowest level of the model.

    Returns
    -------
    result: :class:`xarray.DataArray`
        The extrapolated temperatures at the provided pressure levels.
    """
    R_d = 287.04  # dry air gas constant
    g_inv = 1 / 9.80616  # inverse of gravity
    alpha = 0.0065 * R_d * g_inv

    tstar = data.isel({lev_dim: -1}, drop=True) * (1 + alpha * (ps / p_sfc - 1))
    hgt = phi_sfc * g_inv
    t0 = tstar + 0.0065 * hgt
    tplat = xr.apply_ufunc(np.minimum, 298, t0, dask='parallelized')

    tprime0 = xr.where((2000 <= hgt) & (hgt <= 2500),
                       0.002 * ((2500 - hgt) * t0 + ((hgt - 2000) * tplat)),
                       np.nan)
    tprime0 = xr.where(2500 < hgt, tplat, tprime0)

    alnp = xr.where(hgt < 2000, alpha * np.log(lev / ps),
                    R_d * (tprime0 - tstar) / phi_sfc * np.log(lev / ps))
    alnp = xr.where(tprime0 < tstar, 0, alnp)

    return tstar * (1 + alnp + (0.5 * (alnp**2)) + (1 / 6 * (alnp**3)))


def _geo_height_extrapolate(t_bot, lev, p_sfc, ps, phi_sfc):
    r"""This helper function extrapolates geopotential height below ground using
    the ECMWF formulation described in `Vertical Interpolation and Truncation
    of Model-Coordinate Data <http://dx.doi.org/10.5065/D6HX19NH>`__ by
    Trenberth, Berry, & Buja [NCAR/TN-396, 1993]. Specifically equation 15 is
    used:

    .. math::
        \Phi = \Phi_s - R_d T_* ln \frac{p}{p_s} \left[ 1 + \frac{1}{2}\alpha ln\frac{p}{p_s} + \frac{1}{6} \left( \alpha ln \frac{p}{p_s} \right)^2 \right]

    Parameters
    ----------
    t_bot: :class:`xarray.DataArray`
        Temperature at the lowest (bottom) level of the model.

    lev: int
        The pressure level of interest. Must be in the same units as ``ps`` and ``p_sfc``

    p_sfc: :class:`xarray.DataArray`
        The pressure at the lowest level of the model. Must be in the same units as ``lev`` and ``ps``

    ps : :class:`xarray.DataArray`
        An array of surface pressures. Must be in the same units as ``lev`` and ``p_sfc``

    phi_sfc:
        The geopotential at the lowest level of the model.

    Returns
    -------
    result: :class:`xarray.DataArray`
        The extrapolated geopotential height in geopotential meters at the provided pressure levels.
    """
    R_d = 287.04  # dry air gas constant
    g_inv = 1 / 9.80616  # inverse of gravity
    alpha = 0.0065 * R_d * g_inv

    tstar = t_bot * (1 + alpha * (ps / p_sfc - 1))
    hgt = phi_sfc * g_inv
    t0 = tstar + 0.0065 * hgt

    alph = xr.where((tstar <= 290.5) & (t0 > 290.5),
                    R_d / phi_sfc * (290.5 - tstar), alpha)

    alph = xr.where((tstar > 290.5) & (t0 > 290.5), 0, alph)
    tstar = xr.where((tstar > 290.5) & (t0 > 290.5), 0.5 * (290.5 + tstar),
                     tstar)

    tstar = xr.where((tstar < 255), 0.5 * (tstar + 255), tstar)

    alnp = alph * np.log(lev / ps)
    return hgt - R_d * tstar * g_inv * np.log(
        lev / ps) * (1 + 0.5 * alnp + 1 / 6 * alnp**2)


def _vertical_remap_extrap(new_levels, lev_dim, data, output, pressure, ps,
                           variable, t_bot, phi_sfc):
    """A helper function to call the appropriate extrapolation function based
    on the user's inputs.

    Parameters
    ----------
    new_levels: array-like
        The desired pressure levels for extrapolation in Pascals.

    lev_dim: str
        The name of the vertical dimension.

    data: :class:`xarray.DataArray`
        The data to extrapolate

    output: :class:`xarray.DataArray`
        An array to hold the output data

    pressure: :class:`xarray.DataArray`
        The pressure at the lowest level of the model. Must be in the same units as ``lev`` and ``ps``

    ps : :class:`xarray.DataArray`
        An array of surface pressures. Must be in the same units as ``lev`` and ``p_sfc``

    variable : str, optional
        String representing what variable is extrapolated below surface level.
        Temperature extrapolation = "temperature". Geopotential height
        extrapolation = "geopotential". All other variables = "other". If
        "other", the value of ``data`` at the lowest model level will be used
        as the below ground fill value. Required if extrapolate is True.

    t_bot: :class:`xarray.DataArray`
        Temperature at the lowest (bottom) level of the model.

    phi_sfc:
        The geopotential at the lowest level of the model.

    Returns
    -------
    output: :class:`xarray.DataArray`
        A DataArray containing the data after extrapolation.
    """

    sfc_index = pressure[lev_dim].argmax()  # index of the model surface
    p_sfc = pressure.isel({lev_dim: sfc_index},
                          drop=True)  # extract pressure at lowest level

    if variable == 'temperature':
        output = output.where(
            output.plev <= p_sfc,
            _temp_extrapolate(data, lev_dim, output.plev, p_sfc, ps, phi_sfc))
    elif variable == 'geopotential':
        output = output.where(
            output.plev <= p_sfc,
            _geo_height_extrapolate(t_bot, output.plev, p_sfc, ps, phi_sfc))
    else:
        output = output.where(output.plev <= p_sfc,
                              data.isel({lev_dim: sfc_index}, drop=True))

    return output


def interp_hybrid_to_pressure(data: xr.DataArray,
                              ps: xr.DataArray,
                              hyam: xr.DataArray,
                              hybm: xr.DataArray,
                              p0: float = 100000.,
                              new_levels: np.ndarray = __pres_lev_mandatory__,
                              lev_dim: str = None,
                              method: str = 'linear',
                              extrapolate: bool = False,
                              variable: str = None,
                              t_bot: xr.DataArray = None,
                              phi_sfc: xr.DataArray = None) -> xr.DataArray:
    """Interpolate and extrapolate data from hybrid-sigma levels to isobaric
    levels. Keeps attributes (i.e. metadata) of the input data in the output as
    default.

    Notes
    -----
    ACKNOWLEDGEMENT: We'd like to thank to `Brian Medeiros <https://github.com/brianpm>`__,
    `Matthew Long <https://github.com/matt-long>`__, and `Deepak Cherian <https://github.com/dcherian>`__
    at NSF NCAR for their great contributions since the code implemented here is mostly
    based on their work.

    Parameters
    ----------
    data : :class:`xarray.DataArray`
        Multidimensional data array of hybrid-sigma levels and has a ``lev_dim`` coordinate.

    ps : :class:`xarray.DataArray`
        A multi-dimensional array of surface pressures (Pa), same time/space shape as data.

    hyam, hybm : :class:`xarray.DataArray`
        One-dimensional arrays containing the hybrid A and B coefficients. Must have the same
        dimension size as the ``lev_dim`` dimension of data.

    p0 : float, optional
        Scalar numeric value equal to surface reference pressure (Pa). Defaults to 100000 Pa.

    new_levels : ndarray, optional
        A one-dimensional array of output pressure levels (Pa). If not given, the mandatory
        list of 21 pressure levels is used.

    lev_dim : str, optional
        String that is the name of level dimension in data. Defaults to "lev".

    method : str, optional
        String that is the interpolation method; can be either "linear" or "log". Defaults to "linear".

    extrapolate : bool, optional
        If True, below ground extrapolation for ``variable`` will be done using
        an `ECMWF formulation <http://dx.doi.org/10.5065/D6HX19NH>`__. Defaults
        to False.

    variable : str, optional
        String representing what variable is extrapolated below surface level.
        Temperature extrapolation = "temperature". Geopotential height
        extrapolation = "geopotential". All other variables = "other". If
        "other", the value of ``data`` at the lowest model level will be used
        as the below ground fill value. Required if extrapolate is True.

    t_bot : :class:`xarray.DataArray`, optional
        Temperature in Kelvin at the lowest layer of the model. Not necessarily
        the same as surface temperature. Required if ``extrapolate`` is True
        and ``variable`` is not ``'other'``

    phi_sfc: :class:`xarray.DataArray`, optional
        Geopotential in J/kg at the lowest layer of the model. Not necessarily
        the same as surface geopotential. Required if ``extrapolate`` is True
        and ``variable`` is not ``'other'``.

    Returns
    -------
    output : :class:`xarray.DataArray`
        Interpolated data with isobaric levels

    See Also
    --------
    Related NCL Functions:
    `vinth2p <https://www.ncl.ucar.edu/Document/Functions/Built-in/vinth2p.shtml>`__,
    `vinth2p_ecmwf <https://www.ncl.ucar.edu/Document/Functions/Built-in/vinth2p_ecmwf.shtml>`__
    """

    # Check inputs
    if (extrapolate and (variable is None)):
        raise ValueError(
            "If `extrapolate` is True, `variable` must be provided.")

    if variable in ['geopotential', 'temperature'] and (t_bot is None or
                                                        phi_sfc is None):
        raise ValueError(
            "If `variable` is 'geopotential' or 'temperature', both `t_bot` and `phi_sfc` must be provided"
        )

    if (variable not in ['geopotential', 'temperature', 'other', None]):
        raise ValueError(
            "The value of `variable` is " + variable +
            ", but the accepted values are 'temperature', 'geopotential', 'other', or None."
        )

    # Determine the level dimension and then the interpolation axis
    if lev_dim is None:
        try:
            lev_dim = data.cf["vertical"].name
        except Exception:
            raise ValueError(
                "Unable to determine vertical dimension name. Please specify the name via `lev_dim` argument."
            )

    try:
        func_interpolate = _func_interpolate(method)
    except ValueError as vexc:
        raise ValueError(vexc.args[0])

    interp_axis = data.dims.index(lev_dim)

    # Calculate pressure levels at the hybrid levels
    pressure = _pressure_from_hybrid(ps, hyam, hybm, p0)  # Pa

    # Make pressure shape same as data shape
    pressure = pressure.transpose(*data.dims)

    ###############################################################################
    # Workaround
    #
    # For the issue with metpy's xarray interface:
    #
    # `metpy.interpolate.interpolate_1d` had "no implementation found for
    # 'numpy.apply_along_axis'" issue for cases where the input is
    # xarray.Dataarray and has more than 3 dimensions (e.g. 4th dim of `time`).

    # Use dask.array.core.map_blocks instead of xarray.apply_ufunc and
    # auto-chunk input arrays to ensure using only Numpy interface of
    # `metpy.interpolate.interpolate_1d`.

    # # Apply vertical interpolation
    # # Apply Dask parallelization with xarray.apply_ufunc
    # output = xr.apply_ufunc(
    #     _vertical_remap,
    #     data,
    #     pressure,
    #     exclude_dims=set((lev_dim,)),  # Set dimensions allowed to change size
    #     input_core_dims=[[lev_dim], [lev_dim]],  # Set core dimensions
    #     output_core_dims=[["plev"]],  # Specify output dimensions
    #     vectorize=True,  # loop over non-core dims
    #     dask="parallelized",  # Dask parallelization
    #     output_dtypes=[data.dtype],
    #     dask_gufunc_kwargs={"output_sizes": {
    #         "plev": len(new_levels)
    #     }},
    # )

    # If an unchunked Xarray input is given, chunk it just with its dims
    if data.chunks is None:
        data_chunk = dict([
            (k, v) for (k, v) in zip(list(data.dims), list(data.shape))
        ])
        data = data.chunk(data_chunk)

    # Chunk pressure equal to data's chunks
    pressure = pressure.chunk(data.chunks)

    # Output data structure elements
    out_chunks = list(data.chunks)
    out_chunks[interp_axis] = (new_levels.size,)
    out_chunks = tuple(out_chunks)
    # ''' end of boilerplate

    from dask.array.core import map_blocks
    output = map_blocks(
        _vertical_remap,
        func_interpolate,
        new_levels,
        pressure.data,
        data.data,
        interp_axis,
        chunks=out_chunks,
        dtype=data.dtype,
        drop_axis=[interp_axis],
        new_axis=[interp_axis],
    )

    # End of Workaround
    ###############################################################################

    output = xr.DataArray(output, name=data.name, attrs=data.attrs)

    # Set output dims and coords
    dims = [
        data.dims[i] if i != interp_axis else "plev" for i in range(data.ndim)
    ]

    # Rename output dims. This is only needed with above workaround block
    dims_dict = {output.dims[i]: dims[i] for i in range(len(output.dims))}
    output = output.rename(dims_dict)

    coords = {}
    for (k, v) in data.coords.items():
        if k != lev_dim:
            coords.update({k: v})
        else:
            coords.update({"plev": new_levels})

    output = output.transpose(*dims).assign_coords(coords)

    if extrapolate:
        output = _vertical_remap_extrap(new_levels, lev_dim, data, output,
                                        pressure, ps, variable, t_bot, phi_sfc)

    return output


def interp_sigma_to_hybrid(data: xr.DataArray,
                           sig_coords: xr.DataArray,
                           ps: xr.DataArray,
                           hyam: xr.DataArray,
                           hybm: xr.DataArray,
                           p0: float = 100000.,
                           lev_dim: str = None,
                           method: str = 'linear') -> xr.DataArray:
    """Interpolate data from sigma to hybrid coordinates.  Keeps the attributes
    (i.e. meta information) of the input data in the output as default.

    Parameters
    ----------
    data : :class:`xarray.DataArray`
        Multidimensional data array, which holds sigma levels and has a ``lev_dim`` coordinate.

    sig_coords : :class:`xarray.DataArray`
        A one-dimensional array of sigma coordinates of ``lev_dim`` of ``data``.

    ps : :class:`xarray.DataArray`
        A multi-dimensional array of surface pressures (Pa), same time/space shape as data.

    hyam, hybm : :class:`xarray.DataArray`
        One-dimensional arrays containing the hybrid A and B coefficients. Must have the same
        dimension as the output hybrid levels.

    p0 : float, optional
        Scalar numeric value equal to surface reference pressure (Pa). Defaults to 100000 Pa.

    lev_dim : str, optional
        String that is the name of level dimension in data. Defaults to "lev".

    method : str, optional
        String that is the interpolation method; can be either "linear" or "log". Defaults to "linear".

    Returns
    -------
    output : :class:`xarray.DataArray`
        Interpolated data with hybrid levels

    See Also
    --------
    Related NCL Function:
    `sigma2hybrid <https://www.ncl.ucar.edu/Document/Functions/Built-in/sigma2hybrid.shtml>`__
    """

    # Determine the level dimension and then the interpolation axis
    if lev_dim is None:
        try:
            lev_dim = data.cf["vertical"].name
        except Exception:
            raise ValueError(
                "Unable to determine vertical dimension name. Please specify the name via `lev_dim` argument.'"
            )

    try:
        func_interpolate = _func_interpolate(method)
    except ValueError as vexc:
        raise ValueError(vexc.args[0])

    # Calculate sigma levels at the hybrid levels
    sigma = _sigma_from_hybrid(ps, hyam, hybm, p0)  # Pa

    non_lev_dims = list(data.dims)
    if (data.ndim > 1):
        non_lev_dims.remove(lev_dim)
        data_stacked = data.stack(combined=non_lev_dims).transpose()
        sigma_stacked = sigma.stack(combined=non_lev_dims).transpose()

        h_coords = sigma_stacked[0, :].copy()

        output = data_stacked[:, :len(hyam)].copy()

        for idx, (d, s) in enumerate(zip(data_stacked, sigma_stacked)):
            output[idx, :] = xr.DataArray(_vertical_remap(
                func_interpolate, s.data, sig_coords.data, d.data),
                                          dims=[lev_dim])

        # Make output shape same as data shape
        output = output.unstack().transpose(*data.dims)
    else:
        h_coords = sigma

        output = data[:len(hyam)].copy()
        output[:len(hyam)] = xr.DataArray(_vertical_remap(
            func_interpolate, sigma.data, sig_coords.data, data.data),
                                          dims=[lev_dim])

    # Set output dims and coords
    output = output.rename({lev_dim: 'hlev'})
    output = output.assign_coords({"hlev": h_coords.data})

    return output


def interp_multidim(
        data_in: supported_types,
        lat_out: np.ndarray,
        lon_out: np.ndarray,
        lat_in: np.ndarray = None,
        lon_in: np.ndarray = None,
        cyclic: bool = False,
        missing_val: np.number = None,
        method: str = "linear",
        fill_value: typing.Union[str, np.number] = np.nan) -> supported_types:
    """Multidimensional interpolation of variables. Uses ``xarray.interp`` to
    perform interpolation. Will not perform extrapolation by default, returns
    missing values if any surrounding points contain missing values.

    .. warning::
        The output data type may be promoted to that of the coordinate data.

    Parameters
    ----------
    data_in : :class:`xarray.DataArray`, ndarray
        Data array with data to be interpolated and associated coords. If
        it is a np array, then ``lat_in`` and ``lon_in`` must be provided. Length must
        be coordinated with given coordinates.

    lat_out: ndarray
        List of latitude coordinates to be interpolated to.

    lon_out: ndarray
        List of longitude coordinates to be interpolated to.

    lat_in: ndarray
        List of latitude coordinates corresponding to ``data_in``. Must be
        given if ``data_in`` is not an xarray.

    lon_in: ndarray
        List of longitude coordinates corresponding to ``data_in``. Must be
        given if ``data_in`` is not an xarray.

    cyclic: bool, optional
        Set as true if lon values are cyclical but do not fully wrap around
        the globe
        (0, 1.5, 3, ..., 354, 355.5) Default is false

    missing_val : :class:`np.number`, optional
        Provide a number to represent missing data. Alternative to using ``np.nan``

    method: str, optional
        Provide specific method of interpolation. Default is "linear"
        “linear” or “nearest” for multidimensional array

    fill_value: str, optional
        Set as 'extrapolate' to allow extrapolation of data. Default is
        no extrapolation.

    Returns
    -------
    data_out : ndarray, :class:`xarray.DataArray`
       Returns the same type of object as input ``data_in``. However, the type of
       the data in the array may be promoted to that of the coordinates. Shape
       will be the same as input array except for last two dimensions which will
       be equal to the coordinates given in ``data_out``.

    Examples
    --------
    >>> import xarray as xr
    >>> import numpy as np
    >>> import geocat.comp
    >>> data = np.asarray([[1, 2, 3, 4, 5, 99],
    ...                   [2, 4, 6, 8, 10, 12]])
    >>> lat_in = [0, 1]
    >>> lon_in = [0, 50, 100, 250, 300, 350]
    >>> data_in = xr.DataArray(data,
    ...                        dims=['lat', 'lon'],
    ...                        coords={'lat':lat_in,
    ...                                'lon': lon_in})
    >>> data_out = xr.DataArray(dims=['lat', 'lon'],
    ...                         coords={'lat': [0, 1],
    ...                                 'lon': [0, 50, 360]})
    >>> do = interp_multidim(data_in,
    ...                      [0, 1],
    ...                      [0, 50, 360],
    ...                      cyclic=True,
    ...                      missing_val=99)
    >>> print(do)
    <xarray.DataArray (lat: 2, lon: 3)>
    array([[ 1.,  2., 99.],
       [ 2.,  4., 99.]])
    Coordinates:
      * lat      (lat) int64 0 1
      * lon      (lon) int64 0 50 360

    See Also
    --------
    Related External Functions:
    `xarray.DataArray.interp <https://docs.xarray.dev/en/stable/generated/xarray.DataArray.interp.html>`__,
    `cartopy.util.add_cyclic_point <https://scitools.org.uk/cartopy/docs/latest/reference/generated/cartopy.util.add_cyclic_point.html>`__

    Related NCL Function:
    `NCL linint2 <https://www.ncl.ucar.edu/Document/Functions/Built-in/linint2.shtml>`__
    """
    # check for xarray/numpy
    if not isinstance(data_in, xr.DataArray):
        if lat_in is None or lon_in is None:
            raise ValueError(
                "Argument lat_in and lon_in must be provided if data_in is not an xarray"
            )
        data_in = xr.DataArray(data_in,
                               dims=['lat', 'lon'],
                               coords={
                                   'lat': lat_in,
                                   'lon': lon_in
                               })

    output_coords = {
        data_in.dims[-1]: lon_out,
        data_in.dims[-2]: lat_out,
    }

    data_in_modified = _pre_interp_multidim(data_in, cyclic, missing_val)
    data_out = data_in_modified.interp(output_coords,
                                       method=method,
                                       kwargs={'fill_value': fill_value})
    data_out_modified = _post_interp_multidim(data_out, missing_val=missing_val)

    return data_out_modified

In [ ]:
cluster.close()

In [ ]:
client.close()